# Detect explicit, suggestive and violent content using Amazon Rekognition

This notebook provides a walkthrough of [content moderation APIs](https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html) in Amazon Rekognition. You can quickly identify inappropriate content in your video and image libraries.

## References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_DetectModerationLabels.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartContentModeration.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetContentModeration.html

## Check dependencies setup

In [2]:
%store -r setup_dependencies_passed

In [3]:
%store -r setup_dependencies_passed

try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK] Dependencies correctly set up")

[OK] Dependencies correctly set up


## Setup notebook

In [4]:
import boto3
import sagemaker
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

In [5]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name
rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

## Get image and upload to S3

In [6]:
!mkdir -p ./tmp
!curl -s -o ./tmp/weapon0.jpg "https://collection.nam.ac.uk/images/960/102000-102999/102958.jpg"
!curl -s -o ./tmp/weapon1.jpg "https://observer.com/wp-content/uploads/sites/2/2020/02/56161324_434979100380682_5588907875608035328_n-e1580760304964.png"

In [7]:
temp_folder = "tmp/"
image_names = [
    "weapon0.jpg",
    "weapon1.jpg"
]

for image in image_names:
    s3.upload_file(
        f"{temp_folder}{image}",
        bucket,
        image
    )

display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": image_names[0]})))
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": image_names[1]})))

# Call Rekognition to Detect Unsafe Objects in the Image
Call Amazon Rekognition to detect unsafe content in the image: https://docs.aws.amazon.com/rekognition/latest/dg/moderation.html


In [15]:
detectModerationLabelsResponse_1 = rekognition.detect_moderation_labels(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": image_names[1],
        }
    }
)

# Review JSON Response from Rekognition Moderation API

In the JSON response below, you will see Moderation Labels, confidence score and additional information.


In [16]:
display(detectModerationLabelsResponse_1)

{'ModerationLabels': [{'Confidence': 57.48310089111328,
   'Name': 'Weapon Violence',
   'ParentName': 'Violence'},
  {'Confidence': 57.48310089111328, 'Name': 'Violence', 'ParentName': ''}],
 'ModerationModelVersion': '5.0',
 'ResponseMetadata': {'RequestId': '139f0763-c9a9-4dac-acdf-f85b1a604874',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '139f0763-c9a9-4dac-acdf-f85b1a604874',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '202',
   'date': 'Tue, 15 Nov 2022 18:05:08 GMT'},
  'RetryAttempts': 0}}

# Display list of detected moderation labels

In [17]:
for label in detectModerationLabelsResponse_1["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))

- Weapon Violence (Confidence: 57.48310089111328)
  - Parent: Violence
- Violence (Confidence: 57.48310089111328)
  - Parent: 


# Release Resources

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>